<a href="https://colab.research.google.com/github/talinm23/ML/blob/main/parsing_pdfs_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In order to run these codes (parsing pdfs step 1 and 2), you can run them in a Google Colab notebook.
# To run fast, you should get connected to a GPU in the Colab environment.
# In the Colab secrets, create and copy your HUGGING_FACE_TOKEN and MONGO_URI (MongoDB URI) to the Secrets section.

In [ ]:
import pandas as pd
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab_Notebooks/pdfs/TheOnlyCancerPatientCure_PDF_Download.pdf'

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2

def extract_sentences(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

    sentences = text.split('. ') # Basic sentence splitting
    return sentences

sentences = extract_sentences(file_path)


In [ ]:
sentences_endline_removed = pd.Series(sentences).str.replace('\n', '')

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


In [ ]:
# Uncomment for new runs if needed:
dataset_embedded = pd.Series(sentences_endline_removed).apply(get_embedding)

In [ ]:
dataset = pd.DataFrame(dataset_embedded)
dataset.rename({0: 'embedding_'}, axis=1, inplace=True)

In [ ]:
#dataset_new = pd.DataFrame(dataset)
dataset['sentences'] = sentences_endline_removed

In [ ]:
# save the csv file:
file_path_to_save = '/content/drive/My Drive/Colab_Notebooks/csv_saved/'
dataset.to_csv(file_path_to_save+'dataset_embedded.csv', index=False)
